In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [2]:
# lista nazw plików
files = ["AOS.csv"]# "AMT_data.csv", "AMZN_data.csv", "ANDV_data.csv", "ANSS_data.csv", "ANTM_data.csv", "AON_data.csv", "AOS_data.csv", "APA_data.csv", "APC_data.csv", "APD_data.csv", "APH_data.csv", "APTV_data.csv", "ARE_data.csv", "ARNC_data.csv", "ATVI_data.csv", "AVB_data.csv", "AVGO_data.csv", "AVY_data.csv", "AWK_data.csv", "AXP_data.csv", "AYI_data.csv", "AZO_data.csv", "A_data.csv", "ADP_data.csv", "ADSK_data.csv", "ADS_data.csv", "AEE_data.csv", "AEP_data.csv", "AES_data.csv", "AET_data.csv", "AFL_data.csv", "AGN_data.csv", "AIG_data.csv", "AIV_data.csv", "AIZ_data.csv", "AJG_data.csv", "AKAM_data.csv", "ALB_data.csv", "ALGN_data.csv", "ALK_data.csv", "ALLE_data.csv", "ALL_data.csv", "ALXN_data.csv", "AMAT_data.csv", "AMD_data.csv", "AME_data.csv", "AMGN_data.csv", "AMG_data.csv", "AAL_data.csv", "AAPL_data.csv", "AAP_data.csv", "ABBV_data.csv", "ABC_data.csv", "ABT_data.csv", "ACN_data.csv", "ADBE_data.csv", "ADI_data.csv", "ADM_data.csv"]

K = 50
Procent = K/1259*100
print("Procent przewidywanych próbek: %.2f" % (Procent))

# pętla przez wszystkie pliki
for file in files:
    print("File:", file)
    
    # wczytanie danych
    dt = pd.read_csv(file)
    dt = dt.drop(columns=['date', 'Name'])
    
    # dodanie kolumny prediction
    dt['prediction'] = dt[['open']].shift(-K)
    
    # wejściowe dane
    X = np.array(dt.drop(['prediction'], 1))
    X = preprocessing.scale(X)
    X_dni = X[-K:]
    X = X[:-K]
    
    # wyjściowe dane
    y = np.array(dt['prediction'])
    y = y[:-K]
    
    # podział danych na zbiór treningowy i testowy
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # model
    mlp = MLPRegressor(random_state=1, max_iter=500)
    
    # wyszukiwanie najlepszych parametrów
    parametry = {
    'learning_rate_init': [0.1, 0.01, 0.001], 
    'hidden_layer_sizes': [20, 40, 60, 80, 100], 
    'solver': ['adam', 'lbfgs', 'sgd'], 
    'learning_rate': ['constant', 'adaptive']
    }
    clf = GridSearchCV(mlp, parametry)
    clf.fit(X_train, y_train)
    print("Best params: ", clf.best_params_)
    mlp.fit(X_train, y_train)

    # Oceń dokładność modelu
    print("Dokladność (train): ", mlp.score(X_train, y_train)*100, '%')
    print("Dokladność (test): ", mlp.score(X_test, y_test)*100, '%')

    # Dokonaj predykcji na kolejnych K wierszach
    prediction_dni = mlp.predict(X_dni)
    print("Predykcje dla kolejnych K wierszy: ", prediction_dni)

Procent przewidywanych próbek: 3.97
File: AMP_data.csv
